In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [6]:
import pandas as pd

df = pd.read_csv('./변화율merged_data.csv')
df

,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,wti,brent,gold,vix
0,2024-05-14,371.04,0.91%,0.32%,0.35%,-0.06%,-0.44%,-0.11%,-0.28%,1.32%,0.69%,-0.99%,8.37%
1,2024-05-13,370.98,-1.13%,-0.21%,-0.20%,-0.24%,-0.44%,-0.52%,-0.73%,-0.20%,0.69%,-0.36%,8.37%
2,2024-05-10,371.08,-0.69%,0.32%,0.49%,0.01%,0.10%,0.31%,0.15%,0.47%,0.36%,0.78%,-2.38%
3,2024-05-09,368.84,-0.27%,0.85%,-0.35%,0.86%,0.50%,0.17%,0.17%,1.20%,0.71%,-0.08%,-1.74%
4,2024-05-08,374.09,0.14%,0.44%,-1.63%,-0.61%,0.20%,0.40%,0.03%,-0.73%,-0.73%,-0.30%,-1.93%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,2017-04-21,280.05,-0.13%,-0.15%,1.03%,0.03%,-0.49%,-0.36%,-0.10%,-0.69%,0.11%,0.03%,-5.22%
1735,2017-04-20,277.76,-0.03%,0.85%,-0.01%,0.04%,0.09%,-0.41%,-0.90%,-3.27%,-3.57%,-0.83%,3.54%
1736,2017-04-19,276.49,0.20%,-0.58%,0.07%,-0.81%,0.85%,0.01%,-0.31%,-0.76%,-0.85%,0.17%,-1.64%
1737,2017-04-18,278.23,0.84%,-0.55%,0.35%,-0.79%,-0.30%,0.85%,1.30%,-0.38%,-0.95%,0.26%,-8.15%


In [7]:
for column in df.columns:
    if column != '날짜':
        # 쉼표와 퍼센트 기호를 제거하고 숫자 형태로 변환
        df[column] = df[column].apply(lambda x: str(x).replace(',', '').replace('%', '') if isinstance(x, str) else x)
        # 문자열을 실수형으로 변환하고, 퍼센트 값을 실제 비율로 변환
        df[column] = df[column].astype(float) / 100

In [10]:
# 날짜 칼럼을 datetime 객체로 변환
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

# 타깃 변수 만들어주기
# 단순히 다음 날 오르면 1, 하락하면 0
df['Target'] = (df['kospi200'].shift(-1) > df['kospi200']).astype(int)

In [11]:
import numpy as np

feature_importance = {
    'dow': 0.325531,
    'ssec': 0.238900,
    'nikkei': 0.195132,
    'brent': 0.076208,
    'kosdaq': 0.006516,
    'cny_krw': -0.000277,
    'usd_krw': -0.020725,
    'gold': -0.030892,
    'wti': -0.032822,
    'jpy_krw': -0.172221,
    'vix': -0.223214
}

# 피쳐 중요도 값을 배열 형태로 정의
feature_importance_arr = np.array(list(feature_importance.values()))

# 피쳐 가중치 적용
for feature, importance in feature_importance.items():
    if feature in df.columns:
        df[feature] *= importance

df.head()

,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,wti,brent,gold,vix,Target
날짜,,,,,,,,,,,,,
2024-05-14,3.7104,3.863701e-07,0.000339,0.000133,-0.000034,-1.889913e-06,-8.440190e-11,-0.000083,0.000014,0.000040,-9.447725e-06,0.004170,0
2024-05-13,3.7098,-4.797783e-07,-0.000223,-0.000076,-0.000137,-1.889913e-06,-3.989908e-10,-0.000217,-0.000002,0.000040,-3.435536e-06,0.004170,1
2024-05-10,3.7108,-2.929620e-07,0.000339,0.000187,0.000006,4.295256e-07,2.378599e-10,0.000044,0.000005,0.000021,7.443662e-06,-0.001186,0
2024-05-09,3.6884,-1.146373e-07,0.000901,-0.000133,0.000491,2.147628e-06,1.304393e-10,0.000050,0.000013,0.000041,-7.634525e-07,-0.000867,1
2024-05-08,3.7409,5.944156e-08,0.000466,-0.000621,-0.000348,8.590513e-07,3.069160e-10,0.000009,-0.000008,-0.000042,-2.862947e-06,-0.000962,0


In [12]:
# 피처와 타겟을 분리합니다
features = df.drop('Target', axis=1)
target = df['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)

In [13]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(GRU(units=20))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측

17/17 [==============================] - 1s 4ms/step


In [16]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.46257197696737046
